In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install koalas

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import numpy as np
import databricks.koalas as ks

print('pandas version: %s' % pd.__version__)

print('numpy version: %s' % np.__version__)

print('koalas version: %s' % ks.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

pandas version: 1.4.2
numpy version: 1.21.5
koalas version: 1.8.2
pyarrow version: 7.0.0
pyspark version: 3.3.2.dev0


In [0]:
koalas_data = ks.read_parquet("/FileStore/tables/yellow_tripdata_2023_01.parquet")

koalas_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

In [0]:
def read_file_parquet(df=None):
    return ks.read_parquet("/FileStore/tables/yellow_tripdata_2023_01.parquet")
  
def count(df=None):
    return len(df)

def count_index_length(df=None):
    return len(df.index)

def mean(df):
    return df.fare_amount.mean()

def standard_deviation(df):
    return df.fare_amount.std()

def mean_of_sum(df):
    return (df.fare_amount + df.tip_amount).mean()

def sum_columns(df):
    x = df.fare_amount + df.tip_amount
    x.to_pandas()
    return x

def mean_of_product(df):
    return (df.fare_amount * df.tip_amount).mean()

def product_columns(df):
    x = df.fare_amount * df.tip_amount
    x.to_pandas()
    return x

def value_counts(df):
    val_counts = df.fare_amount.value_counts()
    val_counts.to_pandas()
    return val_counts
  
def complicated_arithmetic_operation(df):
    theta_1 = df.start_lon
    phi_1 = df.start_lat
    theta_2 = df.end_lon
    phi_2 = df.end_lat
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180) ** 2
           + np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180) ** 2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2)
    ret.to_pandas()
    return ret
  
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.start_lon
    phi_1 = df.start_lat
    theta_2 = df.end_lon
    phi_2 = df.end_lat
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180) ** 2
           + np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180) ** 2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 
    return ret.mean()
  
def groupby_statistics(df):
    gb = df.groupby(by='passenger_count').agg(
      {
        'fare_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    )
    gb.to_pandas()
    return gb
  
other = ks.DataFrame(groupby_statistics(koalas_data).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(df.merge(other.spark.hint("broadcast"), left_index=True, right_index=True))

def join_data(df, other):
    ret = df.merge(other.spark.hint("broadcast"), left_index=True, right_index=True)
    ret.to_pandas()
    return ret

In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=koalas_benchmarks, name='read file')
benchmark(count, df=koalas_data, benchmarks=koalas_benchmarks, name='count')
benchmark(count_index_length, df=koalas_data, benchmarks=koalas_benchmarks, name='count index length')
benchmark(mean, df=koalas_data, benchmarks=koalas_benchmarks, name='mean')
benchmark(standard_deviation, df=koalas_data, benchmarks=koalas_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=koalas_data, benchmarks=koalas_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=koalas_data, benchmarks=koalas_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=koalas_data, benchmarks=koalas_benchmarks, name='value counts')
#benchmark(complicated_arithmetic_operation, df=koalas_data, benchmarks=koalas_benchmarks, name='complex arithmetic ops')
#benchmark(mean_of_complicated_arithmetic_operation, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of complex arithmetic ops')
benchmark(groupby_statistics, df=koalas_data, benchmarks=koalas_benchmarks, name='groupby statistics')
benchmark(join_count, koalas_data, benchmarks=koalas_benchmarks, name='join count', other=other)
benchmark(join_data, koalas_data, benchmarks=koalas_benchmarks, name='join', other=other)

count took: 0.49636125564575195 seconds
count index length took: 0.38684678077697754 seconds
mean took: 0.8204250335693359 seconds
standard deviation took: 1.0311601161956787 seconds
mean of columns addition took: 1.1440043449401855 seconds
addition of columns took: 5.273139715194702 seconds
mean of columns multiplication took: 1.2608642578125 seconds
multiplication of columns took: 5.653487920761108 seconds
value counts took: 2.1242949962615967 seconds
groupby statistics took: 2.3534011840820312 seconds
join count took: 4.234711408615112 seconds
join took: 13.209994316101074 seconds
Out[4]: 13.209994316101074

In [0]:
koalas_benchmarks

Out[5]: {'duration': [0.49636125564575195,
  0.38684678077697754,
  0.8204250335693359,
  1.0311601161956787,
  1.1440043449401855,
  5.273139715194702,
  1.2608642578125,
  5.653487920761108,
  2.1242949962615967,
  2.3534011840820312,
  4.234711408615112,
  13.209994316101074],
 'task': ['count',
  'count index length',
  'mean',
  'standard deviation',
  'mean of columns addition',
  'addition of columns',
  'mean of columns multiplication',
  'multiplication of columns',
  'value counts',
  'groupby statistics',
  'join count',
  'join']}

Operations with filtering

In [0]:
expr_filter = (koalas_data.tip_amount >= 1) & (koalas_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]

koalas_filtered = filter_data(koalas_data)

In [0]:
benchmark(count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count')
benchmark(count_index_length, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count index length')
benchmark(mean, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean')
benchmark(standard_deviation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered multiplication of columns')
# benchmark(mean_of_complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered groupby statistics')

other = ks.DataFrame(groupby_statistics(koalas_filtered).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])
benchmark(join_data, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join', other=other)
benchmark(join_count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join count', other=other)

filtered count took: 1.1567907333374023 seconds
filtered count index length took: 0.7659862041473389 seconds
filtered mean took: 1.2688684463500977 seconds
filtered standard deviation took: 1.3339776992797852 seconds
filtered mean of columns addition took: 1.5989813804626465 seconds
filtered addition of columns took: 4.459959983825684 seconds
filtered mean of columns multiplication took: 1.2871794700622559 seconds
filtered multiplication of columns took: 4.186635255813599 seconds
filtered value counts took: 2.959556818008423 seconds
filtered groupby statistics took: 2.545217752456665 seconds
filtered join took: 17.569010019302368 seconds
filtered join count took: 4.511723756790161 seconds
Out[7]: 4.511723756790161

In [0]:
koalas_res_temp = get_results(koalas_benchmarks).set_index('task')
# dask_res_temp = get_results(dask_benchmarks).set_index('task')
# df = pd.concat([koalas_res_temp.duration, dask_res_temp.duration], axis=1, keys=['koalas', 'dask'])

In [0]:

koalas_res_temp

,duration
task,
count,0.496361
count index length,0.386847
mean,0.820425
standard deviation,1.031160
mean of columns addition,1.144004
addition of columns,5.273140
mean of columns multiplication,1.260864
multiplication of columns,5.653488
value counts,2.124295
